In [ ]:
import sys
from pathlib import Path
import matplotlib.pyplot as plt

# add directory with ribasim-repository
sys.path.append("..\\..\\..\\ribasim\\python\\ribasim")
import ribasim

# add directory with ribasim_lumping-repository
sys.path.append("..\\..")
import ribasim_lumping

import pandas as pd
import geopandas as gpd
import numpy as np

from numba.core.errors import NumbaDeprecationWarning
import warnings

warnings.simplefilter("ignore")
pd.options.mode.chained_assignment = None

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# define network name, base dir
network_name            = "tki_zutphen_berkel"
base_dir                = Path("..\\..\\..\\ribasim_lumping_data\\")
# directory D-HYDRO basis calculations
dhydro_basis_dir        = Path(base_dir, network_name, "2_dhydro\\") # zelfde directory, want gebruik zelfde som
# directory D-HYDRO results calculations
dhydro_results_dir      = Path(base_dir, network_name, "2_dhydro\\") # zelfde directory, want gebruik zelfde som
# directory results
results_dir             = Path(base_dir, network_name, "3_ribasim\\")
# batch file used to calculate volume D-Hydro module per GridPoint
dhydro_volume_tool_bat_file = Path(r"c:\Program Files\Deltares\D-HYDRO Suite 2023.03 1D2D\plugins\DeltaShell.Dimr\kernels\x64\dflowfm\scripts\run_dfm_volume_tool.bat")
dhydro_volume_tool_increment = 0.1

In [ ]:
# Create networkanalysis
network = ribasim_lumping.create_ribasim_lumping_network(
    base_dir=base_dir,
    name=network_name,
    dhydro_basis_dir=dhydro_basis_dir,
    dhydro_results_dir=dhydro_results_dir,
    results_dir=results_dir,
    crs=28992
)
# network.export_to_geopackage()

In [ ]:
# Load areas (discharge units: afwaterende eenheden)
areas_dir               = Path(base_dir, network_name, "1_areas")
areas_file_name         = f"{network_name}_afw_eenheden.shp"
areas_id_column         = "GFEIDENT"
network.read_areas(
    areas_file_path=Path(areas_dir, areas_file_name), 
    areas_id_column=areas_id_column
)

In [ ]:
# Read network data and extract all objects (weirs/pumps/laterals/confluences/bifurcations)
network.add_basis_network(
    source_type='dhydro',
    set_name="winter",
    model_dir=Path(dhydro_basis_dir, "afvoergolf"),
    simulation_name="tki_zuthpen_berkel_afvoergolf.dsproj_data",
    dhydro_volume_tool_bat_file=dhydro_volume_tool_bat_file, 
    dhydro_volume_tool_force=True,
    dhydro_volume_tool_increment=dhydro_volume_tool_increment
);
# network.add_basis_network(
#     source_type='dhydro',
#     set_name='zomer', 
#     model_dir=Path(dhydro_basis_dir, "zomer"),
#     simulation_name="tki_zuthpen_berkel_basis.dsproj_data",
#     dhydro_volume_tool_bat_file=dhydro_volume_tool_bat_file, 
#     dhydro_volume_tool_force=True,
#     dhydro_volume_tool_increment=0.1
# );# Read network data and extract all objects (weirs/pumps/laterals/confluences/bifurcations)

In [ ]:
# Select simulation sets and extract all data using xugrid/ugrid
network.add_simulation_set(
    source_type='dhydro',
    set_name="winter",
    model_dir=Path(dhydro_results_dir, "winter"),
    simulation_names=["tki_zuthpen_berkel_basis.dsproj_data"],
    simulation_ts=pd.date_range("2000-01-06 23:00", periods=9, freq="6D"),
);
# network.add_simulation_data(
#     source_type='dhydro',
#     set_name="zomer",
#     model_dir=Path(dhydro_results_dir, "winter"),
#     simulation_names=["tki_zuthpen_berkel_basis.dsproj_data"],
#     simulation_ts=pd.date_range("2000-01-06 23:00", periods=9, freq="6D"),
# );

#### OPZETTEN SIMULATIE

In [ ]:
simulation_code = "test_14f_gebieden"
set_name = 'winter'
starttime = "2020-01-01 00:00:00"
endtime = "2020-02-01 00:00:00"

In [ ]:
# Define locations where the network should be split into Ribasim basins:
network.add_split_nodes(
    weirs=True,
    pumps=True,
    uniweirs=True,
    orifices=True,
    structures_ids_to_include=[
        'kdu_DR80760025', # duiker vispassage bovenstrooms
        'kdu_DR80770020', # duiker afvoergebiedje noordzijde Berkel
        'kdu_DR84930010', # duiker met terugslagklep Zutphen Noorderhaven (parallel aan gemaal)
        'kdu_DR80950033', # duikers voor wijk Leesten
        'kdu_DR80940046', 'kdu_DR80950043', 'kdu_DR80950151', # duikers voor wijk Zuidwijken
        'kdu_DR80950103',
        'kdu_DR80740070', # inlaat twentekanaal
    ], 
    structures_ids_to_exclude=[
        'BCAL_3', 'BCAL_11', # stuwen voor hoogwaterafvoer De Berkel
        'BBypass_Besselink_1', 'BBypass_Besselink_2', 'BBypass_Besselink_3', 'BBypass_Besselink_4', 'BBypass_Besselink_5', # visdrempels vispassage De Berkel
        'kst_ST80950035', # verwarrende stuw ivm afwaterende eenheid (Zutphen: Leesten)
        'kst_ST84930001', # verwarrende stuw ivm afwaterende eenheid (Zutphen: Noorderhaven)
        # 'kst_ST80830045', # weir tussen uitstroompunten Zutphen
        # 'kst_ST80820031', 'kst_ST80820042', # test
    ], 
    edge_ids_to_include=[1801],
);

In [ ]:
network.split_nodes

In [ ]:
split_node_type_conversion = dict(
    weir="TabulatedRatingCurve",
    uniweir="TabulatedRatingCurve",
    pump="Pump",
    culvert="TabulatedRatingCurve",
    edge="ManningResistance",
    orifice="Outlet",
    boundary_connection="ManningResistance"
)
# specify translation for specific split_nodes to ribasim-nodes
split_node_id_conversion = dict(
    kdu_DR80740070='ManningResistance',
    # duikers voor wijk Zuidwijken
    kdu_DR80940046='ManningResistance', 
    kdu_DR80950043='ManningResistance', 
    kdu_DR80950151='ManningResistance', 
)

In [ ]:
network.generate_ribasim_lumping_network(
    simulation_code=simulation_code,
    split_node_type_conversion=split_node_type_conversion,
    split_node_id_conversion=split_node_id_conversion,
);

In [ ]:
# Generate Ribasim model en exporteer naar geopackage
ribasim_model = network.generate_ribasim_model(
    split_node_type_conversion=split_node_type_conversion, 
    split_node_id_conversion=split_node_id_conversion
)
ribasim_model.write(results_dir / "ribasim_model")